# Домашнее задание

В этом домашнем задании вы обучите рекуррентную сеть для генерации текстов в стиле Шекспира.

In [1]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn

## Загрузим данные

Загрузим текстовый файл с пьесами Шекспира.

In [2]:
!wget https://raw.githubusercontent.com/aiedu-courses/rnn_bootcamp/main/shakespeare.txt

--2024-01-16 14:42:11--  https://raw.githubusercontent.com/aiedu-courses/rnn_bootcamp/main/shakespeare.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2024-01-16 14:42:11 (90.4 MB/s) - ‘shakespeare.txt’ saved [1115394/1115394]



*Эта задача аналогична задаче, разобранной на вебинаре, поэтому код мы вам не предоставляем, а предлагаем или написать с нуля, или воспользоваться кодом с вебинара.*

In [3]:
# open text file and read in data as `text`
with open("shakespeare.txt", "r") as f:
    text = f.read()

In [7]:
len(text), type(text)

(1115394, str)

In [8]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))

int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [10]:
len(chars)

65

In [12]:
encoded[:100]

array([44, 40, 22, 56, 33, 47, 54, 40, 33, 40,  4, 36, 16, 32,  3, 45, 36,
        6, 13, 22, 36, 47, 49, 36, 47, 57, 22, 13, 37, 36, 36, 23, 47, 30,
       16, 11, 47,  6, 10, 22, 33, 17, 36, 22,  1, 47, 17, 36, 30, 22, 47,
       46, 36, 47, 56, 57, 36, 30, 24, 52,  3,  3, 27, 34, 34, 32,  3, 25,
       57, 36, 30, 24,  1, 47, 56, 57, 36, 30, 24, 52,  3,  3, 44, 40, 22,
       56, 33, 47, 54, 40, 33, 40,  4, 36, 16, 32,  3,  5, 13, 10])

In [14]:
print(dict(enumerate([7,8,9])))

{0: 7, 1: 8, 2: 9}


## Предобработка данных

Как можно видеть на изображении char-RNN выше, сеть ожидает **one-hot encoded** входа, что означает, что каждый символ преобразуется в целое число (через созданный словарь), а затем преобразуется в вектор-столбец, где только соответствующий ему целочисленный индекс будет иметь значение 1, а остальная часть вектора будет заполнена нулями. Давайте создадим для этого функцию.

In [15]:
def one_hot_encode(arr, n_labels):

    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.0

    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot

In [16]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Создаем мини-батчи (mini-batchs)


Создатдим мини-батчи для обучения. На простом примере они будут выглядеть так:

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>
<br>

Возьмем закодированные символы (переданные как параметр `arr`) и разделим их на несколько последовательностей, заданных параметром `batch_size`. Каждая из наших последовательностей будет иметь длину `seq_length`.

### Создани батчей

**1. Первое, что нам нужно сделать, это отбросить часть текста, чтобы у нас были только полные мини-батчи**

Каждый батч содержит $ N\times M $ символов, где $ N $ - это размер батча (количество последовательностей в батче), а $ M $ - длина `seq_length` или количество шагов в последовательности. Затем, чтобы получить общее количество батчей $ K $, которое мы можем сделать из массива `arr`, нужно разделить длину `arr` на количество символов в батче. Когда мы узнаем количество батчей, можно получить общее количество символов, которые нужно сохранить, из `arr`: $ N * M * K $.

**2. После этого нам нужно разделить `arr` на $N$ батчей**

Это можно сделать с помощью `arr.reshape(size)`, где `size` - это кортеж, содержащий размеры измененного массива. Мы знаем, что нам нужно $ N $ последовательностей в батче, поэтому сделаем его размером первого измерения. Для второго измерения можем использовать «-1» в качестве заполнителя, он заполнит массив соответствующими данными. После этого должен остаться массив $N\times(M * K)$.

**3. Теперь, когда у нас есть этот массив, мы можем перебирать его, чтобы получить наши мини-батчи**

Идея состоит в том, что каждая партия представляет собой окно $ N\times M $ в массиве $ N\times (M * K) $. Для каждого последующего батча окно перемещается на `seq_length`. Мы также хотим создать как входной, так и выходной массивы. Это окно можно сделать с помощью `range`, чтобы делать шаги размером `n_steps` от $ 0 $ до `arr.shape [1]`, общее количество токенов в каждой последовательности. Таким образом, целые числа, которые получены из диапазона, всегда указывают на начало батча, и каждое окно имеет ширину `seq_length`.

In [17]:
def get_batches(int_words, batch_size, seq_length):
    """
    Generates batches from encoded sequence.
    :param int_words: tensor of ints, which represents the text; shape: [batch_size, -1]
    :param batch_size: number of sequences per batch
    :param seq_length: number of encoded chars in a sequence
    :return: generator of pairs (x, y); x_shape, y_shape: [batch_size, seq_length - 1]
    """
    # 1. Truncate text, so there are only full batches
    window_size = seq_length + 1
    batch_size_total = batch_size * window_size
    n_batches = len(int_words) // batch_size_total
    int_words = int_words[: n_batches * batch_size_total]

    # 2. Reshape into batch_size rows
    int_words = int_words.reshape((batch_size, -1))

    # 3. Iterate through the text matrix
    for position in range(0, int_words.shape[1], window_size):
        x = int_words[:, position : position + window_size - 1]
        y = int_words[:, position + 1 : position + window_size]
        yield x, y

### Протестируем

Теперь создадим несколько наборов данных, и проверим, что происходит, когда мы создаем батчи.

In [18]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [19]:
# printing out the first 10 items in a sequence
print("x\n", x[:10, :10])
print("\ny\n", y[:10, :10])

x
 [[44 40 22 56 33 47 54 40 33 40]
 [17 47 33 17 11 47 41 40 37 33]
 [47 40 56 32  3 45 36 33 33 36]
 [30 16 47 30 34 36 17 13 10 56]
 [37 40 30 16 32  3 39 17 30 33]
 [30 22 21 36 18  3 39 36 47 49]
 [36 34 47 30 16 23 47 19 47 24]
 [47 33 17 30 33 47 13 16 37 36]]

y
 [[40 22 56 33 47 54 40 33 40  4]
 [47 33 17 11 47 41 40 37 33 13]
 [40 56 32  3 45 36 33 33 36 22]
 [16 47 30 34 36 17 13 10 56 36]
 [40 30 16 32  3 39 17 30 33 47]
 [22 21 36 18  3 39 36 47 49 40]
 [34 47 30 16 23 47 19 47 24 16]
 [33 17 30 33 47 13 16 37 36 47]]


---
## Зададим архитектуру


<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charRNN.png?raw=1" width=500px>

### Структура модели

В `__init__` предлагаемая структура выглядит следующим образом:
* Создаваём и храним необходимые словари (уже релизовано)
* Определяем слой LSTM, который принимает в качестве параметров: размер ввода (количество символов), размер скрытого слоя `n_hidden`, количество слоев` n_layers`, вероятность drop-out'а `drop_prob` и логическое значение batch_first (True)
* Определяем слой drop-out с помощью drop_prob
* Определяем полносвязанный слой с параметрами: размер ввода `n_hidden` и размер выхода - количество символов
* Наконец, инициализируем веса

Обратите внимание, что некоторые параметры были названы и указаны в функции `__init__`, их нужно сохранить и использовать, выполняя что-то вроде` self.drop_prob = drop_prob`.

---
### Входы-выходы LSTM

Вы можете создать [LSTM layer](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM) следующим образом

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers,
                            dropout=drop_prob, batch_first=True)
```

где `input_siz`e - это количество символов, которые эта ячейка ожидает видеть в качестве последовательного ввода, а `n_hidden` - это количество элементов в скрытых слоях ячейки. Можно добавить drop-out, добавив параметр `dropout` с заданной вероятностью. Наконец, в функции `forward` мы можем складывать ячейки LSTM в слои, используя `.view`.

Также требуется создать начальное скрытое состояние всех нулей:

```python
self.init_hidden()
```

In [20]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()

if train_on_gpu:
    print("Training on GPU!")
else:
    print("No GPU available, training on CPU; consider making n_epochs very small.")

Training on GPU!


In [23]:
class CharRNN(nn.Module):
    def __init__(self, tokens, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr

        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}

        ## TODO: define the LSTM
        self.lstm = nn.LSTM(
            len(self.chars), n_hidden, n_layers, dropout=drop_prob, batch_first=True
        )

        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)

        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))

    def forward(self, x, hidden):
        """Forward pass through the network.
        These inputs are x, and the hidden/cell state `hidden`."""

        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)

        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)

        ## TODO: put x through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden

    def init_hidden(self, batch_size):
        """Initializes hidden state"""
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data

        if train_on_gpu:
            hidden = (
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
            )
        else:
            hidden = (
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
            )

        return hidden

## Обучим модель

Во время обучения нужно установить количество эпох, скорость обучения и другие параметры.

Используем оптимизатор Adam и кросс-энтропию, считаем loss и, как обычно, выполняем back propagation.

Пара подробностей об обучении:
> * Во время цикла мы отделяем скрытое состояние от его истории; на этот раз установив его равным новой переменной * tuple *, потому что скрытое состояние LSTM, является кортежем скрытых состояний.
* Мы используем [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) чтобы избавиться от взрывающихся градиентов.

In [24]:
def train(
    net,
    data,
    epochs=10,
    batch_size=10,
    seq_length=50,
    lr=0.001,
    clip=5,
    val_frac=0.1,
    print_every=10,
):
    """Training a network

    Arguments
    ---------

    net: CharRNN network
    data: text data to train the network
    epochs: Number of epochs to train
    batch_size: Number of mini-sequences per mini-batch, aka batch size
    seq_length: Number of character steps per mini-batch
    lr: learning rate
    clip: gradient clipping
    val_frac: Fraction of data to hold out for validation
    print_every: Number of steps for printing training and validation loss

    """
    net.train()

    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if train_on_gpu:
        net.cuda()

    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)

        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1

            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if train_on_gpu:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size * seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if train_on_gpu:
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(
                        output, targets.view(batch_size * seq_length).long()
                    )

                    val_losses.append(val_loss.item())

                net.train()  # reset to train mode after iterationg through validation data

                print(
                    "Epoch: {}/{}...".format(e + 1, epochs),
                    "Step: {}...".format(counter),
                    "Loss: {:.4f}...".format(loss.item()),
                    "Val Loss: {:.4f}".format(np.mean(val_losses)),
                )

## Определим модель

Теперь мы можем создать модель с заданными гиперпараметрами. Определим размеры мини-батчей.

In [25]:
# define and print the net
n_hidden = 512
n_layers = 2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(65, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=65, bias=True)
)


In [41]:
for i, layer in enumerate(net.parameters()):
  print(i, layer.shape)

0 torch.Size([2048, 65])
1 torch.Size([2048, 512])
2 torch.Size([2048])
3 torch.Size([2048])
4 torch.Size([2048, 512])
5 torch.Size([2048, 512])
6 torch.Size([2048])
7 torch.Size([2048])
8 torch.Size([65, 512])
9 torch.Size([65])


### Установим гиперпараметры

In [26]:
batch_size = 128
seq_length = 100
n_epochs = 20  # start smaller if you are just testing initial behavior

# train the model
train(
    net=net,
    data=encoded,
    epochs=n_epochs,
    batch_size=batch_size,
    seq_length=seq_length,
    lr=0.001,
    print_every=10,
)

Epoch: 1/20... Step: 10... Loss: 3.3573... Val Loss: 3.3892
Epoch: 1/20... Step: 20... Loss: 3.3215... Val Loss: 3.3481
Epoch: 1/20... Step: 30... Loss: 3.3091... Val Loss: 3.3456
Epoch: 1/20... Step: 40... Loss: 3.3358... Val Loss: 3.3357
Epoch: 1/20... Step: 50... Loss: 3.3102... Val Loss: 3.3126
Epoch: 1/20... Step: 60... Loss: 3.2412... Val Loss: 3.2610
Epoch: 1/20... Step: 70... Loss: 3.1970... Val Loss: 3.1825
Epoch: 2/20... Step: 80... Loss: 3.1332... Val Loss: 3.1035
Epoch: 2/20... Step: 90... Loss: 3.0536... Val Loss: 3.0161
Epoch: 2/20... Step: 100... Loss: 2.9751... Val Loss: 2.9260
Epoch: 2/20... Step: 110... Loss: 2.8259... Val Loss: 2.7652
Epoch: 2/20... Step: 120... Loss: 2.7561... Val Loss: 2.7013
Epoch: 2/20... Step: 130... Loss: 2.6321... Val Loss: 2.5856
Epoch: 2/20... Step: 140... Loss: 2.5608... Val Loss: 2.5046
Epoch: 2/20... Step: 150... Loss: 2.5032... Val Loss: 2.4502
Epoch: 3/20... Step: 160... Loss: 2.4612... Val Loss: 2.4100
Epoch: 3/20... Step: 170... Loss:

## Checkpoint

После обучения сохраним модель, чтобы можно было загрузить ее позже. Здесь сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметры скрытого слоя и токены.

In [42]:
# change the name, for saving multiple files
model_name = "rnn_x_epoch.net"

checkpoint = {
    "n_hidden": net.n_hidden,
    "n_layers": net.n_layers,
    "state_dict": net.state_dict(),
    "tokens": net.chars,
}

with open(model_name, "wb") as f:
    torch.save(checkpoint, f)

---
## Делаем предсказания

Теперь, когда мы обучили модель, сделаем предсказание следующих символов! Для предсказания мы передаем последний символ, и сеть предсказывает следующий символ, который мы потом передаем снова на вхол и получаем еще один предсказанный символ и так далее...

Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным символам. Мы можем ограничить число символов, чтобы сделать получаемый предсказанный текст более разумным, рассматривая только некоторые наиболее вероятные символы $K$. Это не позволит сети выдавать нам совершенно абсурдные прогнозы, а также позволит внести некоторый шум и случайность в выбранный текст. Узнать больше [можно здесь](https://pytorch.org/docs/stable/generated/torch.topk.html#torch.topk).

In [43]:
def predict(net, char, h=None, top_k=None):
    """Given a character, predict the next character.
    Returns the predicted character and the hidden state.
    """

    # tensor inputs
    x = np.array([[net.char2int[char]]])
    x = one_hot_encode(x, len(net.chars))
    inputs = torch.from_numpy(x)

    if train_on_gpu:
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    # get the output of the model
    out, h = net(inputs, h)

    # get the character probabilities
    p = F.softmax(out, dim=1).data
    if train_on_gpu:
        p = p.cpu()  # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(net.chars))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return net.int2char[char], h

### Priming и генерирование текста

Нужно задать скрытое состояние, чтобы сеть не генерировала произвольные символы.

In [44]:
def sample(net, size, prime="The", top_k=None):

    if train_on_gpu:
        net.cuda()
    else:
        net.cpu()

    net.eval()  # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)

    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return "".join(chars)

In [45]:
print(sample(net, 1000, top_k=5))

Theme a man aro true that I shall.

KING RICHARD III:
Ay, brings your hands where'st he hours of thy lives.

BUCKINGHAM:
Why, thou sheet so by; be to me, but the were,
And which all hour and the pease is true sour,
That ther we shall be sontial of his poor as thou
constlunting, as I have see thee to this bed the sire
And will be the way.

PAULINA:
Thy lord, the country to thee?

CORIOLANUS:
What is a prayer of the shall what we would tear.

LADY ANNE:
A presate of the prince, but there's a poor
As husband and his bariage to such a merty
But with her heavens thee, what wasto yourself there,
And then, if he speaker with me to the palling, to his son?
When you have do not, by at this some service shall
I was the senting her then are at take of more;
Fet, thone that senst to stay is see and stiff as
to show their tales. To them to his fair of the duke.

BUCKINGHAM:
Ay, sor of to a hory to be the person
Whits and the trancours, and that the sun war stay
By how is to the world.

POMPEY:
That

In [46]:
supernet = CharRNN(chars, n_hidden, n_layers)

In [47]:
train(
    net=supernet,
    data=encoded,
    epochs=n_epochs,
    batch_size=batch_size,
    seq_length=seq_length,
    lr=1,
    print_every=10,
)

Epoch: 1/20... Step: 10... Loss: 812.6558... Val Loss: 848.0257
Epoch: 1/20... Step: 20... Loss: 1769.9945... Val Loss: 1834.5202
Epoch: 1/20... Step: 30... Loss: 1901.9198... Val Loss: 1774.6995
Epoch: 1/20... Step: 40... Loss: 1335.9194... Val Loss: 1288.4412
Epoch: 1/20... Step: 50... Loss: 906.2190... Val Loss: 1010.4716
Epoch: 1/20... Step: 60... Loss: 683.6893... Val Loss: 682.7831
Epoch: 1/20... Step: 70... Loss: 498.5186... Val Loss: 506.0106
Epoch: 2/20... Step: 80... Loss: 429.9596... Val Loss: 450.0662
Epoch: 2/20... Step: 90... Loss: 386.6889... Val Loss: 399.9857
Epoch: 2/20... Step: 100... Loss: 297.9394... Val Loss: 231.7064
Epoch: 2/20... Step: 110... Loss: 270.1555... Val Loss: 255.6065
Epoch: 2/20... Step: 120... Loss: 261.2060... Val Loss: 238.6037
Epoch: 2/20... Step: 130... Loss: 263.0961... Val Loss: 285.7343
Epoch: 2/20... Step: 140... Loss: 283.4082... Val Loss: 281.4579
Epoch: 2/20... Step: 150... Loss: 311.7438... Val Loss: 291.9000
Epoch: 3/20... Step: 160...

KeyboardInterrupt: 